Intro stuff

In [15]:
import tweepy
import json

consumer_key = "U6eIfx0myrDZxymjY6rxFQuH3"
consumer_secret = "bmHWK7d1ktxIa43iOar1eyPs2kG9frLFecv0E6mdYWXex3VCnM"
access_token = "1351114116291842050-Sqx8kbMr7mVneQasWdjkrj9CH4NfI8"
access_token_secret = "VfmTCgDjmPRAQ7vwGMysI6izKnvgPSrRqqO035ZPxDIPT"

In [65]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.keys())
    


dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])


Get first tweets from topic

In [100]:
xlp = api.search("$EXPRS2", count = 100, lang = "sv")
len(xlp["statuses"])

3

In [102]:
for tweet in xlp["statuses"]:
    print(tweet.get("user").get("name"), ":", tweet.get("text"))
    print("################")

triumfen : BAYN, BUSER EXPRS2 ja de finns nästan oändligt med fler exempel, alla förhoppningsbolag har tydligen blivit duktigt… https://t.co/niRp3O0j49
################
Hbgtrader : #exprs2 när man tyckte 15% över natten var en vettig affär för 24h sen....
################
Flowtrading : @P_a_s3 Lång EXPRS2? :)
################


{'created_at': 'Tue Feb 16 19:53:50 +0000 2021',
 'id': 1361765753347735552,
 'id_str': '1361765753347735552',
 'text': 'RT @JessiesPennies: $NBY Hey Alexa what is an example of a golden cross? https://t.co/rhith06q95',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [{'text': 'NBY', 'indices': [20, 24]}],
  'user_mentions': [{'screen_name': 'JessiesPennies',
    'name': 'JessiesPennies',
    'id': 1213885096756834305,
    'id_str': '1213885096756834305',
    'indices': [3, 18]}],
  'urls': [],
  'media': [{'id': 1361764968379645952,
    'id_str': '1361764968379645952',
    'indices': [73, 96],
    'media_url': 'http://pbs.twimg.com/media/EuX2BhfXAAAqzti.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/EuX2BhfXAAAqzti.jpg',
    'url': 'https://t.co/rhith06q95',
    'display_url': 'pic.twitter.com/rhith06q95',
    'expanded_url': 'https://twitter.com/JessiesPennies/status/1361764970413850625/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 1